<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [128]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [129]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head(20)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0
242171,952,2013-01-12 08:50:22,947.0,2013-01-12 08:50:23,953.0,2013-01-12 08:50:23,946.0,2013-01-12 08:50:23,947.0,2013-01-12 08:50:24,...,2013-01-12 08:50:24,953.0,2013-01-12 08:50:24,955.0,2013-01-12 08:50:24,946.0,2013-01-12 08:50:25,947.0,2013-01-12 08:50:25,0
57157,953,2013-01-12 08:50:25,947.0,2013-01-12 08:50:26,946.0,2013-01-12 08:50:26,953.0,2013-01-12 08:50:26,955.0,2013-01-12 08:50:26,...,2013-01-12 08:50:27,953.0,2013-01-12 08:50:27,946.0,2013-01-12 08:50:27,953.0,2013-01-12 08:50:28,1033.0,2013-01-12 08:50:28,0
240201,946,2013-01-12 08:50:28,947.0,2013-01-12 08:50:28,954.0,2013-01-12 08:50:28,953.0,2013-01-12 08:50:29,946.0,2013-01-12 08:50:29,...,2013-01-12 08:50:29,946.0,2013-01-12 08:50:30,956.0,2013-01-12 08:50:30,957.0,2013-01-12 08:50:31,956.0,2013-01-12 08:50:31,0
210686,946,2013-01-12 08:50:31,956.0,2013-01-12 08:50:32,946.0,2013-01-12 08:50:32,946.0,2013-01-12 08:50:33,955.0,2013-01-12 08:50:33,...,2013-01-12 08:50:33,946.0,2013-01-12 08:50:34,946.0,2013-01-12 08:50:35,946.0,2013-01-12 08:50:36,948.0,2013-01-12 08:50:36,0


In [130]:
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [131]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтовcross_val_score(lasso, X, y, cv=3)
with open(r"../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head(20)

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu
37491,img378.imageshack.us
5462,ecologie.nature.free.fr
35425,www.ibcn.intec.ugent.be
30151,kissanime.com
38268,www.carolineconduiteformation.com


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [132]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [133]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947
242171,952,947,953,946,947,946,953,955,946,947
57157,953,947,946,953,955,947,953,946,953,1033
240201,946,947,954,953,946,954,946,956,957,956
210686,946,956,946,946,955,954,946,946,946,948


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [134]:
from scipy.sparse import csr_matrix

In [135]:
csr_matrix?

In [136]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [137]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_val = X[train_len:, :]
    y_train = y[:train_len]
    y_val = y[train_len:]
    
    model = LogisticRegression(C=C, random_state=seed, n_jobs=-1)
    model.fit(X_train, y_train)
    
    y_predict = model.predict_proba(X_val)[:, 1]
    
    return roc_auc_score(y_val, y_predict)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [138]:
X_train_sparse = full_sites_sparse[:idx_split]

In [139]:
X_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

In [140]:
get_auc_lr_valid(X_train_sparse, y_train)

0.9197955574958125

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [141]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [142]:
model = LogisticRegression(random_state=17, n_jobs=-1)
model.fit(X_train_sparse, y_train)

predict = model.predict_proba(full_sites_sparse[idx_split:])[:, 1]
write_to_submission_file(predict, 'result_2.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [143]:
full_df['time_start'] = full_df['time1'].apply(lambda d: int(d.year * 100 + d.month))

In [158]:
full_df.tail(50)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time7,site8,time8,site9,time9,site10,time10,time_start,start_hour,morning
session_id,,,,,,,,,,,,,,,,,,,,,
82748,167,2014-11-04 17:33:52,167,2014-11-04 17:33:53,167,2014-11-04 17:33:54,359,2014-11-04 17:33:54,812,2014-11-04 17:33:54,...,2014-11-04 17:33:57,812,2014-11-04 17:34:00,679,2014-11-04 17:34:12,39,2014-11-04 17:34:25,0.703932,17,False
82749,217,2014-06-02 11:06:44,562,2014-06-02 11:06:44,261,2014-06-02 11:06:44,3897,2014-06-02 11:06:44,1002,2014-06-02 11:06:44,...,2014-06-02 11:06:45,17749,2014-06-02 11:06:46,3897,2014-06-02 11:06:46,261,2014-06-02 11:06:46,0.577432,11,True
82750,1278,2014-05-23 18:14:06,13123,2014-05-23 18:14:22,29,2014-05-23 18:14:22,13123,2014-05-23 18:14:25,1278,2014-05-23 18:14:26,...,2014-05-23 18:14:29,22,2014-05-23 18:14:31,784,2014-05-23 18:14:34,0,NaT,0.552132,18,False
82751,67,2014-11-04 15:49:34,71,2014-11-04 15:49:35,70,2014-11-04 15:49:35,21,2014-11-04 15:49:55,21,2014-11-04 15:49:57,...,2014-11-04 15:49:59,21,2014-11-04 15:50:11,22,2014-11-04 15:50:11,23,2014-11-04 15:50:11,0.703932,15,False
82752,786,2014-11-04 10:32:03,786,2014-11-04 10:32:05,48,2014-11-04 10:32:07,786,2014-11-04 10:32:08,786,2014-11-04 10:32:10,...,2014-11-04 10:32:15,786,2014-11-04 10:32:17,786,2014-11-04 10:32:19,786,2014-11-04 10:32:22,0.703932,10,True
82753,363,2014-08-01 10:36:16,1651,2014-08-01 10:36:16,362,2014-08-01 10:36:16,364,2014-08-01 10:36:16,360,2014-08-01 10:36:16,...,2014-08-01 10:36:17,1652,2014-08-01 10:36:17,362,2014-08-01 10:36:17,1651,2014-08-01 10:36:17,0.628032,10,True
82754,29,2014-05-28 18:25:56,2296,2014-05-28 18:25:56,20649,2014-05-28 18:26:15,20649,2014-05-28 18:26:16,2296,2014-05-28 18:26:17,...,2014-05-28 18:26:35,20649,2014-05-28 18:26:36,29,2014-05-28 18:26:36,2296,2014-05-28 18:26:37,0.552132,18,False
82755,42743,2014-05-27 10:17:58,42741,2014-05-27 10:17:58,42740,2014-05-27 10:17:58,5275,2014-05-27 10:17:58,42742,2014-05-27 10:17:58,...,2014-05-27 10:17:59,42740,2014-05-27 10:18:00,42742,2014-05-27 10:18:00,42740,2014-05-27 10:18:01,0.552132,10,True
82756,780,2014-10-04 11:04:21,780,2014-10-04 11:04:22,782,2014-10-04 11:04:23,780,2014-10-04 11:04:30,782,2014-10-04 11:04:30,...,2014-10-04 11:04:31,780,2014-10-04 11:04:37,780,2014-10-04 11:04:38,780,2014-10-04 11:04:39,0.678632,11,True


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [145]:
scaler = StandardScaler()
scaler.fit(full_df['time_start'].values.reshape(-1, 1))

StandardScaler()

In [146]:
full_df['time_start'] = scaler.transform(full_df['time_start'].values.reshape(-1, 1))

In [147]:
sparse_data = hstack([full_df['time_start'].values.reshape(-1, 1), full_sites_sparse]).tocsr()

In [148]:
full_df['start_hour'] = full_df['time1'].apply(lambda t: t.hour)

In [151]:
sparse_data = hstack([full_df['start_hour'].values.reshape(-1, 1), sparse_data]).tocsr()

In [154]:
full_df['morning'] = full_df['time1'].apply(lambda t: t.hour <= 11)

In [155]:
sparse_data = hstack([full_df['morning'].values.reshape(-1, 1), sparse_data]).tocsr()

In [162]:
sparse_data.shape, y_train.shape

((336358, 48374), (253561,))

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [228]:
get_auc_lr_valid(sparse_data[:idx_split, :], y_train)

0.9550063913755978

In [172]:
model = LogisticRegression(random_state=401, n_jobs=-1)
model.fit(sparse_data[:idx_split], y_train)

predict = model.predict_proba(sparse_data[idx_split:])[:, 1]
write_to_submission_file(predict, 'result_2.csv')

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [173]:
get_auc_lr_valid(sparse_data[:idx_split, :], y_train, C=1)

0.9550063913755978

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [224]:
C_values = np.logspace(-3, 1, 20)
C_values

array([1.00000000e-03, 1.62377674e-03, 2.63665090e-03, 4.28133240e-03,
       6.95192796e-03, 1.12883789e-02, 1.83298071e-02, 2.97635144e-02,
       4.83293024e-02, 7.84759970e-02, 1.27427499e-01, 2.06913808e-01,
       3.35981829e-01, 5.45559478e-01, 8.85866790e-01, 1.43844989e+00,
       2.33572147e+00, 3.79269019e+00, 6.15848211e+00, 1.00000000e+01])

In [239]:
from sklearn.model_selection import cross_val_score

def get_auc_with_validation(X, y, C=1.0, seed=17):
    model = LogisticRegression(C=C, random_state=seed, n_jobs=-1)
    return cross_val_score(model, X, y, cv=5).mean()

In [240]:
for c in C_values:
    print('C =', c, ': ', get_auc_with_validation(sparse_data[:idx_split, :], y_train, C=c))

C = 0.001 :  0.9909094852131745
C = 0.001623776739188721 :  0.9908976536940074
C = 0.0026366508987303583 :  0.990858215607855
C = 0.004281332398719396 :  0.9908384963314754
C = 0.0069519279617756054 :  0.9908266644234693
C = 0.011288378916846888 :  0.9908148327487666
C = 0.018329807108324356 :  0.9907990584008589
C = 0.029763514416313176 :  0.9908187775994707
C = 0.04832930238571752 :  0.9907753966845778
C = 0.07847599703514611 :  0.9907162400997243
C = 0.12742749857031335 :  0.9904677820078394
C = 0.20691380811147903 :  0.9901128414099685
C = 0.3359818286283781 :  0.990144392594354
C = 0.5455594781168515 :  0.9900970638735798
C = 0.8858667904100823 :  0.9895133862488636
C = 1.438449888287663 :  0.9895488772290448
C = 2.3357214690901213 :  0.9896553615236909
C = 3.792690190732246 :  0.9896080347471123
C = 6.158482110660261 :  0.9894897211107981
C = 10.0 :  0.9891150621697614


Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [245]:
final_model = LogisticRegression(C=0.1, random_state=17, n_jobs=-1)
final_model.fit(sparse_data[:idx_split, :], y_train)
predict = final_model.predict_proba(sparse_data[idx_split:])[:, 1]
write_to_submission_file(predict, 'result.csv')